# Hello Registration World!

In [17]:
import itk
from itkwidgets import view, compare, checkerboard
import ipywidgets
import os

**Image registration** finds the *spatial transformation that aligns images in the presence of noise*.

In image registration, we typically identify the two images as the fixed and moving image. Our goal is to find the spatial transformation that makes the moving image align with the fixed image.

First, let's load our **fixed image** and the image we will align to our fixed image, the **moving image**.

In [7]:
fixedUploader = ipywidgets.FileUpload(
    accept='image/*,.nrrd,.mha,.nii',
    multiple=False
)
display(fixedUploader)

# these FileUpload widgets supported up to 10MB files
# https://github.com/jupyter-widgets/ipywidgets/issues/2522

movingUploader = ipywidgets.FileUpload(
    accept='image/*,.nrrd,.mha,.nii',
    multiple=False
)
display(movingUploader)

FileUpload(value={}, accept='image/*,.nrrd,.mha,.nii', description='Upload')

FileUpload(value={}, accept='image/*,.nrrd,.mha,.nii', description='Upload')

In [18]:
# write to temporary image file
# I don't know how to read image from memory in Python
# using FileUpload's syntax for ipywidgets <= 7
tempFilename="./fixed_"+ next(iter(fixedUploader.value))
with open(tempFilename, "wb") as fp:
    fp.write(fixedUploader.data[0])
fixed = itk.imread(tempFilename, itk.F)
os.remove(tempFilename)

tempFilename="./moving_"+ next(iter(movingUploader.value))
with open(tempFilename, "wb") as fp:
    fp.write(movingUploader.data[0])
moving = itk.imread(tempFilename, itk.F)
os.remove(tempFilename)

In [19]:
compare(fixed, moving, ui_collapsed=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

Before registration, the moving image is not aligned with the fixed image.

In [20]:
checkerboard(fixed, moving, pattern=8)

In [21]:
# Register!
registered_moving = itk.elastix_registration_method(fixed, moving)

AttributeError: module 'Elastix' has no attribute 'swig'

The registered moving image is aligned with the fixed image

In [7]:
compare(fixed, registered_moving, ui_collapsed=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [8]:
checkerboard(fixed, registered_moving, pattern=10)

Now, let's do it **in 3D**!

In [9]:
fixed = itk.imread('data/CT_3D_lung_fixed.mha', itk.F)
moving = itk.imread('data/CT_3D_lung_moving.mha', itk.F)

In [12]:
# A human lung at different time points
compare(fixed, moving, link_cmap=True, link_gradient_opacity=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=True, description='cmap'), Checkbox(va…

In [13]:
checkerboard(fixed, moving, mode='z', ui_collapsed=False)

In [15]:
# With the default, conservative parameters, this can take a few minutes
# on a low-compute-power system.
registered = itk.elastix_registration_method(fixed, moving)

In [16]:
checkerboard(fixed, registered, mode='z', ui_collapsed=False)